## Passive Seismic Metadata Standards (Geoscience Australia) v(0.2)

This document describes how corrections for GPS clock-errors and station-orientations are encapsulated in StationXML files.  

### 1. QA/QC Procedure

[HiPerSeis](https://github.com/GeoscienceAustralia/hiperseis) implements elaborate workflows for computing corrections for:

1. [GPS clock errors](https://github.com/GeoscienceAustralia/hiperseis/tree/develop/seismic/gps_corrections)
 
 This workflow outputs simple csv files for each station processed, as follows:
 ```
 net,sta,loc,comp,date,clock_correction
 OA,BM26,,HHZ,2019-05-25T00:00:00,0.040261028
 OA,BM26,,HHZ,2019-05-26T00:00:00,0.269642617
 OA,BM26,,HHZ,2019-05-27T00:00:00,0.518772089
 OA,BM26,,HHZ,2019-05-28T00:00:00,0.787649446
 OA,BM26,,HHZ,2019-05-29T00:00:00,1.076274687
 ...
  ```
 Each row above lists the daily clock-correction (in seconds) for a given station. 
 
 
2. [Station-orientation errors](https://github.com/GeoscienceAustralia/hiperseis/tree/develop/seismic#orientation-analysis)

 This workflow generates orientation correction estimates based on two separate 
 methods: (i) Receiver function ([Wilde-Piórko et al. 2017](https://doi.org/10.1007/s10950-017-9640-x)), (ii) Surface wave polarization ([Doran and Lakse, 2017](https://doi.org/10.1785/0120160165))

 Correction estimates are output in Json format as follows:
 ```json
 {
    "rf": {
        "OA.BK24.": {
            "date_range": [
                "2019-05-30T15:45:33.505000Z",
                "2020-09-13T11:58:30.490000Z"
            ],
            "azimuth_correction": -4.0
        }
        ...
        ...
    }
    "swp": {
        "OA.BK24.": {
            "date_range": [
                "2019-05-26T07:40:04.360000Z",
                "2020-09-11T11:35:53.700000Z"
            ],
            "azimuth_correction": 1.3485540759336345,
            "uncertainty": 3.6205002297639233
        }
        ...
        ...
    }
}
``` 
 Correction estimates from the surface wave polarization method have associated uncertainties, which are currently unavailable for those from the receiver function method.

### 2. Embedding QC/QC Metadata into StationXML Files

Obspy [inventory utilities](https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.Inventory.html) are used to embed the corrections described above within StationXML files as [extra station metadata](https://docs.obspy.org/tutorial/code_snippets/stationxml_custom_tags.html).

Once these QA/QC metadata are embedded, a typical StationXML file looks like as follows:

```xml
<Network code="OA" 
         startDate="2017-09-11T00:00:36" 
         endDate="2018-11-28T23:06:20">
    <Station code="BY22" 
             startDate="2017-10-01T13:58:58" 
             endDate="2018-11-24T21:13:47">
      <GeoscienceAustralia:clock_corrections>[{'0M': [{'2018-07-28T00:00:00.000000 - 2018-07-29T00:00:00.000000': 0.432706767}, {'2018-07-29T00:00:00.000000 - 2018-07-30T00:00:00.000000': 0.768295739}, {'2018-07-30T00:00:00.000000 - 2018-07-31T00:00:00.000000': 1.124155978}, {'2018-07-31T00:00:00.000000 - 2018-08-01T00:00:00.000000': 1.500287483}, {'2018-08-01T00:00:00.000000 - 2018-08-02T00:00:00.000000': 1.896690255}, {'2018-08-02T00:00:00.000000 - 2018-08-03T00:00:00.000000': 2.313364293}, {'2018-08-03T00:00:00.000000 - 2018-08-04T00:00:00.000000': 2.750309598}, {'2018-08-04T00:00:00.000000 - 2018-08-05T00:00:00.000000': 3.207526168}, {'2018-08-05T00:00:00.000000 - 2018-08-06T00:00:00.000000': 3.685014006}, {'2018-08-06T00:00:00.000000 - 2018-08-07T00:00:00.000000': 4.182773109}, {'2018-08-07T00:00:00.000000 - 2018-08-08T00:00:00.000000': 4.700803479}, {'2018-08-08T00:00:00.000000 - 2018-08-09T00:00:00.000000': 5.239105116}, {'2018-08-09T00:00:00.000000 - 2018-08-10T00:00:00.000000': 5.797678019}, {'2018-08-10T00:00:00.000000 - 2018-08-11T00:00:00.000000': 6.376522188}, {'2018-08-11T00:00:00.000000 - 2018-08-12T00:00:00.000000': 6.975637623}, {'2018-08-12T00:00:00.000000 - 2018-08-13T00:00:00.000000': 7.595024326}, {'2018-08-13T00:00:00.000000 - 2018-08-14T00:00:00.000000': 8.234682294}, {'2018-08-14T00:00:00.000000 - 2018-08-15T00:00:00.000000': 8.894611529}, {'2018-08-15T00:00:00.000000 - 2018-08-16T00:00:00.000000': 9.57481203}]}]
      </GeoscienceAustralia:clock_corrections>
      
      <GeoscienceAustralia:rf_orientation_corrections>[{'0M': [{'2017-10-06T08:08:02.844999 - 2018-11-22T16:21:07.070000': 5.0}]}]         
      </GeoscienceAustralia:rf_orientation_corrections>

      <GeoscienceAustralia:swp_orientation_corrections>[{'0M': [{'2017-10-03T06:52:09.700000 - 2018-11-15T01:21:51.440000': [2.373463113875232, 3.271484514337502]}]}]
      </GeoscienceAustralia:swp_orientation_corrections>
      
...
...
```

For each station, `clock_corrections` contain lists of dictionaries with timespan as the key and clock-correction as the value in seconds, for each location code. Similarly, `rf_orientation_corrections` and `swp_orientation_corrections` contain orientation correction estimates. 

### 3. Extracting QA/QC Metadata

QA/QC metadata can be extracted through the Obspy [Inventory](https://docs.obspy.org/tutorial/code_snippets/stationxml_custom_tags.html) interface as follows: 

In [10]:
from obspy import Inventory
from obspy.core.inventory import Network, read_inventory

inv = read_inventory('OA.xml')
sta = inv.networks[0].stations[1]

# Extract corrections from the inventory 
if(hasattr(sta.extra, 'clock_corrections')):
    print('Clock corrections for station {}:\n'.format(sta.code))
    print(sta.extra.clock_corrections)
# end if

if(hasattr(sta.extra, 'rf_orientation_corrections')):
    print('\nRF Orientation correction for station {}:\n'.format(sta.code))
    print(sta.extra.rf_orientation_corrections)
# end if

if(hasattr(sta.extra, 'swp_orientation_corrections')):
    print('\nSWP Orientation correction and uncertainty ' \
          'for station {}:\n'.format(sta.code))
    print(sta.extra.swp_orientation_corrections)
# end if

Clock corrections for station BS25:

AttribDict({'namespace': 'https://github.com/GeoscienceAustralia/hiperseis', 'value': "[{'0M': [{'2018-04-16T00:00:00.000000 - 2018-04-17T00:00:00.000000': -0.916666667}, {'2018-04-17T00:00:00.000000 - 2018-04-18T00:00:00.000000': -0.886904762}, {'2018-04-18T00:00:00.000000 - 2018-04-19T00:00:00.000000': -0.857142857}, {'2018-04-19T00:00:00.000000 - 2018-04-20T00:00:00.000000': -0.827380952}, {'2018-04-20T00:00:00.000000 - 2018-04-21T00:00:00.000000': -0.797619048}, {'2018-04-21T00:00:00.000000 - 2018-04-22T00:00:00.000000': -0.767857143}, {'2018-04-22T00:00:00.000000 - 2018-04-23T00:00:00.000000': -0.738095238}, {'2018-04-23T00:00:00.000000 - 2018-04-24T00:00:00.000000': -0.708333333}, {'2018-04-24T00:00:00.000000 - 2018-04-25T00:00:00.000000': -0.833334188}, {'2018-04-25T00:00:00.000000 - 2018-04-26T00:00:00.000000': -1.133324819}, {'2018-04-26T00:00:00.000000 - 2018-04-27T00:00:00.000000': -0.433329917}, {'2018-04-27T00:00:00.000000 - 2018-04-28T

### 4. Metadata Extractor Script

In addition to accessing metadata programmatically above, a script [export_corrections.py](https://github.com/GeoscienceAustralia/hiperseis/blob/develop/seismic/inventory/export_corrections.py) is provided to extract the metadata into csv files.

#### Requirements

python3, obspy (version 1.1.0), pandas

#### Launch Exporter

The meta-data exporter is launched as follows:

`python export_corrections.py OA.xml meta`

The inputs are: (i) a StationXML file with embedded metadata (ii) output-file base-name

Upon successful completion, two csv files should be produced:

 1. `meta.clock_corrections.csv`, with typical content as follows:
 ```
# GPS clock-corrections grouped by network, station and location
Network,Station,Location,Start-time,End-time,Correction_in_seconds
OA,BS25,0M,2018-04-16T00:00:00.000000,2018-04-17T00:00:00.000000,-0.916666667
OA,BS25,0M,2018-04-17T00:00:00.000000,2018-04-18T00:00:00.000000,-0.886904762
 ...
 ...
 ```
 2. `meta.orientation_corrections.csv`, with typical content as follows:
```
# Orientation corrections are derived from two separate methods: (i) Receiver Function (RF) (ii) Surface-wave Polarization (SWP). Only the latter method provides uncertainty estimates.
Network,Station,Location,Method,Start-time,End-time,Azimuth_correction_in_degrees,Uncertainty±
OA,BS24,0M,RF,2017-10-06T08:08:12.755000,2018-08-17T22:17:31.755000,2.0,
OA,BS25,,RF,2019-12-04T20:15:16.408000,2021-03-19T19:02:55.508000,-1.0,
...
...
```
